<a href="https://colab.research.google.com/github/Pieter-devries/python_playground/blob/main/Query_Favorite_Contents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run me to initialize SDK
Open cell to check individual functions

In [ ]:
!pip install looker_sdk
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import looker_sdk #Note that the pip install required a hyphen but the import is an underscore.
from looker_sdk import models as mdls #This is required for high-level sdk calls
import os #We import os here in order to manage environment variables for the tutorial. You don't need to do this on a local system or anywhere you can more conveniently set environment variables.

import json #This is a handy library for doing JSON work.

from pandas import json_normalize


In [ ]:
# Setup Environment Settings
# Setup Instance/SDK information
os.environ["LOOKERSDK_BASE_URL"] = "https://switchback.cloud.looker.com" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

# Setup User login information
# Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "MrnXgW9T68m6ws55rB4m" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "S4JRGdnWfq2Hb6PC6phRcNfS" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.
print("All environment variables set.")

All environment variables set.


In [ ]:
# Initialize sdk
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

Looker SDK 4.0 initialized successfully.


# Get User List

In [ ]:
# Create User list with email/user_id values
user_list = dict()
users = sdk.all_users()
for user in users:
  if user['email'] and not user['is_disabled']: #removing users that don't have emails setup and aren't disabled
    user_list[user['email']] = user['id']
print(user_list)

{'gsatyanarayana21@gmail.com': '179', 'ismailadeyemi@google.com': '32', 'akashkragarwal@google.com': '5', 'kenellysra@google.com': '20', 'yanzai@google.com': '58', 'christopherav@google.com': '166', 'jayantb@google.com': '43', 'vbari@google.com': '160', 'bariksourav@google.com': '88', 'raghaveni@google.com': '157', 'mannybhatia@google.com': '147', 'sayanbiswas@google.com': '85', 'alet@google.com': '156', 'ryan.bouquet@looker.com': '1', 'theobrowne@google.com': '82', 'edgardol@google.com': '164', 'mrkitten@google.com': '182', 'heliocampos@google.com': '91', 'scomerford@google.com': '134', 'jcort@google.com': '46', 'gowriswari@google.com': '180', 'pieterdevries@google.com': '21', 'pieterdevries+changed@google.com': '26', 'dharmadhikari@google.com': '141', 'nourad@google.com': '8', 'mebadi@google.com': '90', 'artellis@google.com': '92', 'joespla@google.com': '81', 'gadugoyyala@google.com': '25', 'gadugoyyala@looker.com': '53', 'pushkargoel@google.com': '16', 'boydahardy@google.com': '177'

In [ ]:
# Create Dashboard list with title/content_metadata_id values
dashboard_list = dict()
dashboards = sdk.all_dashboards()
for dashboard in dashboards:
  if dashboard['content_metadata_id']: #removing dashboards without content_metadata_id
    dashboard_list[dashboard['title']] = dashboard['content_metadata_id']
print(dashboard_list)

{'test': '18', 'Embed Dashboard': '29', 'Users Test Dashboard': '32', 'Users Test Age and Country': '33', 'Users Count test': '36', 'Video Analytics': '49', 'Map indicating No. of cities/state': '56', 'India AQI analysis (2015-2020)': '59', 'download test': '60', 'Month Wise Distribution': '61', 'Top 10 Polluted Cities': '62', "Covid🦠 effect on AQI😷 in India's Megacities": '70', 'Map indicating Avg. AQI/state': '76', 'Sample': '77', '228925690 bug repro': '82', 'Orders_DB': '83', 'orderitems_DB': '85', 'Video_Series': '86', 'D2': '108', 'D1': '109', 'Best Episode': '112', 'Real Time AQI': '116', 'Suggested Data by Avg Rating': '122', 'test_timezone': '127', 'T1': '129', 'T2': '255', 'Sneaker shares on StockX': '142', 'Sneaker Brand Insights': '143', 'Sneaker Model Insight': '144', 'Rating Overview': '156', 'Categories Overview': '159', "Let's talk air pollution": '164', '⚠️': '165', 'Category Analysis': '178', 'Boardgame Analysis': '181', "Welcome to Mukul's Thesis": '197', 'Initiative

In [ ]:
# Check all dashboards if the current user has favorited them or not
favorited_content = dict()

def retrieve_favorite_content(): #Create a function so we can loop it for each user
  for k,v in dashboard_list.items():
    response = sdk.search_content_favorites(content_metadata_id=v)
    if response:
      email = [k for k,v in user_list.items() if v == response[0]['user_id']][0] #Change user_id into email for data readability
      dashboard = [k for k,v in dashboard_list.items() if v == response[0]['content_metadata_id']][0] #Change content_metadata_id into dashboard name for data readability
      if email in favorited_content: #If we already have a favorited content, we need to append the new content
        favorited_content[email].append(dashboard)
      else: #If we don't have any favorited content yet, we add the new content as a list
        favorited_content[email] = []
        favorited_content[email].append(dashboard)



In [ ]:
for k,v in user_list.items():
  sdk.auth.logout() #This is required if you stopped the sdk mid-way
  sudo_auth = sdk.login_user(v)
  sdk.auth.login_user(v)
  retrieve_favorite_content()
print(favorited_content)

/usr/local/lib/python3.7/dist-packages/looker_sdk/sdk/api40/methods.py:432: UserWarning: login_user behavior changed significantly in 21.4.0. See https://git.io/JOtH1
  "login_user behavior changed significantly in 21.4.0. See https://git.io/JOtH1"


KeyboardInterrupt: ignored